In [ ]:
!git clone --depth 1 https://github.com/roguehunter7/CornerSentinal

Cloning into 'CornerSentinal'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 42 (delta 0), reused 39 (delta 0), pack-reused 0
Receiving objects: 100% (42/42), 247.52 MiB | 38.03 MiB/s, done.
Updating files: 100% (39/39), done.


In [ ]:
!pip install ultralytics filterpy scikit-image lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.1/660.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=4c39d33b123127eccd0a52bda92ca2193afd9eb739c486520ad8cc24132cce58
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628966 sha256=f7fd8dca6b0b75874bee9dd1f65987b2486d38dc103ad45bee939d44c5b2cd83
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built filterpy lap


In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from time import time
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8x.pt')

# Open the video file
video_path = "/content/CornerSentinal/test_images/leftside.mp4"
cap = cv2.VideoCapture(video_path)

# Define the constants for speed calculation
VIDEO_FPS = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Video FPS: {VIDEO_FPS}")
FACTOR_KM = 3.6
LATENCY_FPS = int(cap.get(cv2.CAP_PROP_FPS)) / 2
print(f"Dynamic Latency FPS: {LATENCY_FPS}")

# Store the track history for each vehicle
track_history = defaultdict(list)
stationary_timers = defaultdict(float)

# Define the output video file path with MP4 format
output_video_path = "/content/CornerSentinal/test_images/leftside_out.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for H.264 compression
out = cv2.VideoWriter(output_video_path, fourcc, VIDEO_FPS, (int(cap.get(3)), int(cap.get(4))))

# Function to calculate Euclidean distance
def calculate_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

# Function to calculate speed using Euclidean distance
def calculate_speed(distances, factor_km=FACTOR_KM, latency_fps=LATENCY_FPS):
    average_speed = (np.mean(distances) * factor_km * latency_fps)
    return average_speed

# Function to generate 9-bit binary code based on conditions
def generate_binary_code(class_id, speed, is_stationary, is_wrong_side):
    binary_code = ['0'] * 9
    binary_code[0] = '1'  # start bit

    # Stationary bit
    binary_code[1] = '1' if is_stationary else '0'

    # Replace class id section
    if class_id == 2:  # Motorcycle
        binary_code[2:5] = '001'
    elif class_id == 3:  # Car
        binary_code[2:5] = '010'
    elif class_id in [5, 7]:  # Bus or Truck
        binary_code[2:5] = '011'

    # Wrong side warning bit
    binary_code[5] = '1' if is_wrong_side else '0'

    # Replace speed section
    if speed > 60:  # Overspeed Vehicle
        binary_code[6:8] = '11'
    elif 40 <= speed < 60:
        binary_code[6:8] = '10'
    elif 1.5 <= speed < 40:
        binary_code[6:8] = '01'

    binary_code[8] = '1'  # stop bit
    return ''.join(binary_code)

# Function to check if the motion is towards the camera
def is_motion_towards_camera(prev_box, curr_box):
    return curr_box[0] < prev_box[0]

# Placeholder function to simulate displaying binary data
def display_warning_message(frame, binary_code):
    warning_message = "Warning: " + binary_code
    cv2.putText(frame, warning_message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, tracker='botsort.yaml', classes=[2, 3, 5, 7])
        annotated_frame = results[0].plot()

        # Check if the results are not None and contain boxes
        if results[0].boxes.id is not None:
            # Get the boxes, ids, and confidences
            boxes = results[0].boxes.xywh.cpu().numpy().astype(int)
            class_id = results[0].boxes.cls.cpu().numpy().astype(int)
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)

            # Plot the tracks and calculate/display speeds using Euclidean distance
            for i, box in enumerate(boxes):
                x, y, w, h = box
                track = track_history[track_ids[i]]
                track.append((float(x + w / 2), float(y + h / 2)))  # x, y center point

                # Check if the motion is towards the camera
                if len(track) >= 2 and is_motion_towards_camera(track[-2], track[-1]):
                    # Calculate Euclidean distance
                    distances = [calculate_distance(track[j], track[j + 1]) for j in range(len(track) - 1)]

                    # Calculate speed using Euclidean distance
                    if len(distances) > 1:
                        speed = calculate_speed(distances)

                        # Check if the vehicle is stationary
                        is_stationary = speed < 1.0  # You can adjust the threshold as needed

                        # Update stationary timer
                        stationary_timers[track_ids[i]] = time() if not is_stationary else stationary_timers[track_ids[i]]

                        # Check if the vehicle has been stationary for more than 10 seconds
                        if time() - stationary_timers[track_ids[i]] > 10.0:
                            is_stationary = True

                        # Check if the vehicle is on the wrong side
                        is_wrong_side = False  # Replace with your logic

                        # Generate binary code based on conditions
                        if is_stationary and is_wrong_side:
                            binary_code = generate_binary_code(class_id[i], speed, is_stationary, is_wrong_side)
                        elif class_id[i] in [5, 7]:
                            binary_code = generate_binary_code(class_id[i], speed, is_stationary, is_wrong_side)
                        elif speed > 60 or is_wrong_side:
                            binary_code = generate_binary_code(class_id[i], speed, is_stationary, is_wrong_side)

                        # Display warning message at the top left of the frame
                        display_warning_message(annotated_frame, binary_code)

                        # Display speed on the frame (corrected scaling)
                        cv2.putText(annotated_frame, f"Speed: {speed:.2f} km/h", (int(x), int(y) - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Display the annotated frame
            out.write(annotated_frame)  # Save the frame to the output video

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video writer and close the display window
out.release()
cap.release()
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
0: 384x640 1 car, 30.7ms
Speed: 3.1ms preprocess, 30.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.7ms
Speed: 3.0ms preprocess, 31.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.1ms
Speed: 2.0ms preprocess, 31.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 32.0ms
Speed: 2.3ms preprocess, 32.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.6ms
Speed: 2.0ms preprocess, 31.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 30.7ms
Speed: 1.9ms preprocess, 30.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 30.9ms
Speed: 1.8ms preprocess, 30.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 31.8ms
Speed: 2.0ms preprocess, 31.8ms inference, 1.6ms postprocess pe

In [ ]:
!rm -rf '/content/leftside.mp4'
import os
# Set up NVIDIA GPU device for FFmpeg
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Convert leftside_out.mp4 to leftside.mp4 using GPU-accelerated FFmpeg
os.system(f"ffmpeg -hwaccel cuda -i '/content/CornerSentinal/test_images/leftside_out.mp4' -vcodec hevc_nvenc '/content/leftside.mp4'")

0

In [ ]:
from google.colab import files
files.download('/content/leftside.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm -rf '/content/rightside.mp4'

import os

# Set up NVIDIA GPU device for FFmpeg
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Convert hbfootage_out.mp4 to footage.mp4 using GPU-accelerated FFmpeg
os.system(f"ffmpeg -hwaccel cuda -i '/content/CornerSentinal/test_images/rightside_out.mp4' -vcodec hevc_nvenc '/content/rightside.mp4'")

In [ ]:
from google.colab import files
files.download('/content/rightside.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>